In [1]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [89]:
#Trazendo os dataframes necessários da análise
dataframeFato = pd.read_excel("Fato.xlsx")
dataframeProduto = pd.read_excel("dProdutos.xlsx")

In [90]:
dataframeFato.head()

,cod_pdv,data_emissao,Origem,Cod_sku,quantidade,valor,Vendedor,status_nf,cod_movimento,cod_cidade,cod_bairro
0,182,2019-12-02,Televenda,988,4,160.699997,Vendedor 24,Entregue,51,2990,3971
1,302,2019-12-02,Televenda,982,2,74.919998,Vendedor 24,Entregue,51,2990,3962
2,302,2019-12-02,Televenda,978,4,70.970001,Vendedor 24,Entregue,51,2990,3962
3,302,2019-12-02,Televenda,10530,1,48.939999,Vendedor 24,Entregue,51,2990,3962
4,302,2019-12-02,Televenda,1695,1,49.330002,Vendedor 24,Entregue,51,2990,3962


In [119]:
#Realizando o join entre as duas tabelas
dataframeJoin = pd.merge(dataframeFato, dataframeProduto, left_on='Cod_sku', right_on='cod_sku').dropna()

dataframeJoin["Categoria_Embalagem"] = dataframeJoin["nom_tipo_prod"] + "_" + dataframeJoin["Embalagem"]

In [120]:
dataframeJoin.head()

,cod_pdv,data_emissao,Origem,Cod_sku,quantidade,valor,Vendedor,status_nf,cod_movimento,cod_cidade,cod_bairro,cod_sku,nom_sku,fator_hl,cod_tipo_prod,nom_tipo_prod,Embalagem,Categoria_Embalagem
0,182,2019-12-02,Televenda,988,4,160.699997,Vendedor 24,Entregue,51,2990,3971,988,Cerveja P,0.072,1,CERVEJA,600 ML,CERVEJA_600 ML
1,302,2019-12-02,Televenda,988,2,80.339996,Vendedor 24,Entregue,51,2990,3962,988,Cerveja P,0.072,1,CERVEJA,600 ML,CERVEJA_600 ML
2,1893,2019-12-02,Vendedor,988,4,160.699997,Vendedor 12,Entregue,51,3229,4215,988,Cerveja P,0.072,1,CERVEJA,600 ML,CERVEJA_600 ML
3,3100,2019-12-02,Canal Digital,988,2,84.980003,Vendedor 4,Entregue,51,3229,4224,988,Cerveja P,0.072,1,CERVEJA,600 ML,CERVEJA_600 ML
4,4630,2019-12-02,Televenda,988,2,80.339996,Vendedor 25,Entregue,51,2990,3979,988,Cerveja P,0.072,1,CERVEJA,600 ML,CERVEJA_600 ML


In [153]:
#Grouping dataframe by order and transforming it from pandas object to list, keeping only category names 
grouped = dataframeJoin.groupby("cod_pdv") 
Agrupamento1 = grouped['Categoria_Embalagem'].apply(list).to_list() 

Agrupamento2 = grouped['nom_tipo_prod'].apply(list).to_list() 

Agrupamento3 = grouped['nom_sku'].apply(list).to_list() 

In [154]:
#Deleting orders one item only in order to increase both confidence and lift
for lista in Agrupamento1:
    if len(lista) == 1:
        Agrupamento1.remove(lista)
for lista in Agrupamento2:
    if len(lista) == 1:
        Agrupamento2.remove(lista)
for lista in Agrupamento3:
    if len(lista) == 1:
        Agrupamento3.remove(lista)

In [135]:
#One hot encoding

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
teArray = te.fit(Agrupamento1).transform(Agrupamento1)
teDataframe1 = pd.DataFrame(teArray, columns=te.columns_)

teDataframe1.head()

,AGUA_500 ML,BEBIDAS QUENTES_Caçulinha,BEBIDAS QUENTES_Lata,BEBIDAS QUENTES_Long Neck,CERVEJA_1 Litro,CERVEJA_300 ML,CERVEJA_600 ML,CERVEJA_Barril,CERVEJA_Lata,CERVEJA_Long Neck,...,REFRIGERANTE_2 Litros,"REFRIGERANTE_2,5 Litros",REFRIGERANTE_3 Litros,REFRIGERANTE_300 ML,REFRIGERANTE_500 ML,REFRIGERANTE_600 ML,REFRIGERANTE_Bag,REFRIGERANTE_Caçulinha,REFRIGERANTE_Lata,SUCO_1 Litro
0,False,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,False,False,True
1,False,False,False,False,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,True,True,False,True,True,...,True,False,False,False,True,False,False,True,False,False
3,False,False,False,False,True,True,True,False,True,True,...,False,False,False,False,True,False,False,True,False,False
4,False,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,True,False,False,True,False


In [136]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(teDataframe1, min_support = 0.3, use_colnames = True) 
frequent_itemsets.sort_values(by=['support'], ascending = True).head(10)

#Support is defined as frequency that both items occur together in the basket divided by total of transactions in the dataset
#Since there are too many orders with no category variability, support is low

,support,itemsets
12,0.300599,"(CERVEJA_300 ML, CERVEJA_600 ML)"
13,0.311252,"(CERVEJA_300 ML, CERVEJA_Lata)"
10,0.322570,"(CERVEJA_1 Litro, REFRIGERANTE_2 Litros)"
14,0.332557,"(CERVEJA_600 ML, CERVEJA_Lata)"
7,0.333222,"(CERVEJA_300 ML, CERVEJA_1 Litro)"
11,0.360186,"(CERVEJA_1 Litro, REFRIGERANTE_Caçulinha)"
16,0.375166,"(REFRIGERANTE_Caçulinha, CERVEJA_Lata)"
9,0.376831,"(CERVEJA_1 Litro, CERVEJA_Lata)"
17,0.379161,"(REFRIGERANTE_2 Litros, REFRIGERANTE_Caçulinha)"
15,0.381491,"(REFRIGERANTE_2 Litros, CERVEJA_Lata)"


In [137]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
#rules
rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

#Lift > 1 means that there's a significative association rule

,antecedents,consequents,support,confidence,lift
11,(CERVEJA_600 ML),(CERVEJA_300 ML),0.300599,0.597617,1.392741
10,(CERVEJA_300 ML),(CERVEJA_600 ML),0.300599,0.700543,1.392741
3,(CERVEJA_600 ML),(CERVEJA_1 Litro),0.394141,0.783587,1.366161
2,(CERVEJA_1 Litro),(CERVEJA_600 ML),0.394141,0.687174,1.366161
1,(CERVEJA_1 Litro),(CERVEJA_300 ML),0.333222,0.580963,1.353929
0,(CERVEJA_300 ML),(CERVEJA_1 Litro),0.333222,0.776571,1.353929
12,(CERVEJA_300 ML),(CERVEJA_Lata),0.311252,0.725369,1.257361
13,(CERVEJA_Lata),(CERVEJA_300 ML),0.311252,0.539527,1.257361
17,(CERVEJA_Lata),(REFRIGERANTE_2 Litros),0.381491,0.661281,1.184549
16,(REFRIGERANTE_2 Litros),(CERVEJA_Lata),0.381491,0.683363,1.184549


In [138]:
#One hot encoding

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
teArray = te.fit(Agrupamento2).transform(Agrupamento2)
teDataframe2 = pd.DataFrame(teArray, columns=te.columns_)

teDataframe2.head()

,AGUA,BEBIDAS QUENTES,CERVEJA,ENERGETICO,ISOTONICO,REFRIGERANTE,SUCO
0,False,False,True,False,True,True,True
1,False,False,True,False,False,False,False
2,False,False,True,False,False,True,False
3,False,False,True,False,False,True,False
4,False,False,True,False,False,True,False


In [139]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(teDataframe2, min_support = 0.1, use_colnames = True) 
frequent_itemsets.sort_values(by=['support'], ascending = True).head(10)

#Support is defined as frequency that both items occur together in the basket divided by total of transactions in the dataset
#Since there are too many orders with no category variability, support is low

,support,itemsets
3,0.101864,"(CERVEJA, ISOTONICO)"
5,0.115513,"(REFRIGERANTE, ISOTONICO)"
1,0.119174,(ISOTONICO)
4,0.741012,"(CERVEJA, REFRIGERANTE)"
0,0.846205,(CERVEJA)
2,0.894141,(REFRIGERANTE)


In [141]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
#rules
rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

#Lift > 1 means that there's a significative association rule

,antecedents,consequents,support,confidence,lift
4,(REFRIGERANTE),(ISOTONICO),0.115513,0.129188,1.084028
5,(ISOTONICO),(REFRIGERANTE),0.115513,0.969274,1.084028
0,(CERVEJA),(ISOTONICO),0.101864,0.120378,1.010096
1,(ISOTONICO),(CERVEJA),0.101864,0.854749,1.010096
3,(REFRIGERANTE),(CERVEJA),0.741012,0.828742,0.979363
2,(CERVEJA),(REFRIGERANTE),0.741012,0.875688,0.979363


In [155]:
#One hot encoding

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
teArray = te.fit(Agrupamento3).transform(Agrupamento3)
teDataframe3 = pd.DataFrame(teArray, columns=te.columns_)

teDataframe3.head()

,Agua A,Bebida Quente A,Bebida Quente C,Bebida Quente E,Cerveja A,Cerveja AA,Cerveja AB,Cerveja AC,Cerveja AD,Cerveja AE,...,Refrigerante W,Refrigerante X,Refrigerante Y,Refrigerante Z,Suco A,Suco B,Suco C,Suco D,Suco E,Suco G
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [160]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(teDataframe3, min_support = 0.25, use_colnames = True) 
frequent_itemsets.sort_values(by=['support'], ascending = True).head(10)

#Support is defined as frequency that both items occur together in the basket divided by total of transactions in the dataset
#Since there are too many orders with no category variability, support is low

,support,itemsets
17,0.252663,"(Refrigerante E, Cerveja AH)"
14,0.253662,(Refrigerante W)
0,0.260652,(Cerveja AG)
15,0.261318,"(Cerveja M, Cerveja AH)"
12,0.262983,(Refrigerante M)
18,0.265313,"(Refrigerante K, Cerveja AH)"
20,0.271971,"(Cerveja P, Cerveja M)"
22,0.280626,"(Refrigerante B, Refrigerante E)"
2,0.281625,(Cerveja AO)
6,0.282956,(Cerveja Q)


In [161]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
#rules
rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

#Lift > 1 means that there's a significative association rule

,antecedents,consequents,support,confidence,lift
15,(Refrigerante E),(Refrigerante B),0.280626,0.605169,1.903588
14,(Refrigerante B),(Refrigerante E),0.280626,0.882723,1.903588
3,(Cerveja R),(Cerveja AH),0.298269,0.815287,1.898543
2,(Cerveja AH),(Cerveja R),0.298269,0.694574,1.898543
9,(Cerveja O),(Cerveja M),0.364514,0.961370,1.885088
8,(Cerveja M),(Cerveja O),0.364514,0.714752,1.885088
19,(Refrigerante K),(Refrigerante N),0.435087,0.763881,1.601324
18,(Refrigerante N),(Refrigerante K),0.435087,0.912073,1.601324
11,(Cerveja M),(Cerveja P),0.271971,0.533290,1.488850
10,(Cerveja P),(Cerveja M),0.271971,0.759294,1.488850
